### Libraries

In [ ]:
!pip install mediapipe

In [3]:
import numpy as np
import pandas as pd
import cv2
import mediapipe as mp
import os
import glob
import csv
import scipy.io as sio
from math import sin, cos
import warnings
warnings.filterwarnings('ignore')
#from google.colab.patches import cv2_imshow

In [4]:
import numpy as np
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
#from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

import warnings
warnings.filterwarnings('ignore')
#from google.colab.patches import cv2_imshow

In [5]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Models Retrieval

In [5]:
with open('final_pitch_mdl.pkl', 'rb') as f:
  mdl_pitch = pickle.load(f)

with open('final_yaw_mdl.pkl', 'rb') as f:
  mdl_yaw= pickle.load(f)

with open('final_roll_mdl.pkl', 'rb') as f:
  mdl_roll = pickle.load(f)

### Live Testing with OpenCV

In [6]:
mp_face_mesh = mp.solutions.face_mesh

In [32]:
#drawing function
def draw_axis(image, pitch,yaw,roll, tdx=None, tdy=None, size = 100):

    yaw = -yaw

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(image, (int(tdx), int(tdy)), (int(x1),int(y1)),(255,0,0),3)
    cv2.line(image, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(image, (int(tdx), int(tdy)), (int(x3),int(y3)),(0,0,255),3)

In [38]:
cap = cv2.VideoCapture(0)
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
videoWriter = cv2.VideoWriter('head_pose_test_video.mp4', fourcc, 30.0, (640,480))

with mp_face_mesh.FaceMesh(static_image_mode=True) as face_mesh:

  while cap.isOpened():
    #read image
    ret, frame = cap.read()
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    #get image shape
    shape = image.shape
    #convert image from BGR to RGB and process
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    if results.multi_face_landmarks != None:
      #loop over the faces
      for face in results.multi_face_landmarks:
        
        #choose the chosen points to normalize
        #nose point 4
        nose_pt_x, nose_pt_y = face.landmark[4].x, face.landmark[4].y
        #forehead point 10
        forehead_pt_x, forehead_pt_y = face.landmark[10].x, face.landmark[10].y

        #un-normalize points to get their location
        nose_pt_x, nose_pt_y = int(nose_pt_x * shape[1]), int(nose_pt_y * shape[0])
        forehead_pt_x, forehead_pt_y = int(forehead_pt_x * shape[1]), int(forehead_pt_y * shape[0])

        #calculate the distance
        nose_pt = np.array([nose_pt_x, nose_pt_y])
        forehead_pt = np.array([forehead_pt_x, forehead_pt_y])
        distance = np.linalg.norm(nose_pt - forehead_pt)

        face_row = list(np.array([[((lm.x * shape[1]) - nose_pt_x)/distance, ((lm.y * shape[0]) - nose_pt_y)/distance] for lm in face.landmark]).flatten())

        arr_row = np.array((face_row)).reshape(1, -1)

        pitch = mdl_pitch.predict(arr_row)
        yaw = mdl_yaw.predict(arr_row)
        roll = mdl_roll.predict(arr_row)

        draw_axis(image, pitch, yaw, roll, tdx=nose_pt_x, tdy=nose_pt_y, size = 100)

        image = cv2.flip(image,1)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        cv2.putText(image, f'Pitch:{pitch}', (20,90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(image, f'Yaw:{yaw}', (20,120), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
        cv2.putText(image, f'Roll:{roll}', (20,150), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1, cv2.LINE_AA)

        videoWriter.write(image)

        cv2.imshow('Raw Webcam Feed', image)
      
      if cv2.waitKey(1) & 0xFF == ord('q'):
        break
videoWriter.release()
cap.release()
cv2.destroyAllWindows()